In [18]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [46]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# === Load saved scaler parameters ===
scaler_mean = np.load("scaler_mean_combined.npy")
scaler_scale = np.load("scaler_scale_combined.npy")

# Recreate the scaler
scaler = StandardScaler()
scaler.mean_ = scaler_mean
scaler.scale_ = scaler_scale
scaler.var_ = scaler_scale ** 2

from tensorflow.keras.models import load_model
residual_cnn_model = load_model("final_combined_cnn_model.h5")
cnn_gru_model = load_model("final_combined_cnn_gru_model.h5")
pure_cnn_model = load_model("final_pure_cnn_model.h5")
cnn_transormer_model = load_model("final_cnn_transformer_model.h5")

In [47]:
import pandas as pd

data_BBH_df = pd.read_csv("./BBH/processed.csv", header=None)
X = data_BBH_df.values.astype(float)

print(len(data_BBH_df.index))

y = [1 for i in range(len(data_BBH_df.index))]

X = scaler.transform(X)

X = np.expand_dims(X, axis=-1)
print(X.shape)

48
(48, 16384, 1)


In [48]:
pred_residual_cnn = residual_cnn_model.predict(X)
y_pred = (pred_residual_cnn.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by Residual CNN model: {correct}")
print(f"Accuracy of Residual CNN model: {acc:.2f}%")

2/2 [==============================] - 0s 257ms/step
Total no. of original gravitational data samples: 48
No. of correct predictions by Residual CNN model: 40
Accuracy of Residual CNN model: 83.33%


In [49]:
pred_cnn_gru = cnn_gru_model.predict(X)
y_pred = (pred_cnn_gru.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by CNN + GRU model: {correct}")
print(f"Accuracy of CNN + GRU model: {acc:.2f}%")

2/2 [==============================] - 0s 22ms/step
Total no. of original gravitational data samples: 48
No. of correct predictions by CNN + GRU model: 48
Accuracy of CNN + GRU model: 100.00%


In [50]:
pred_pure_cnn = pure_cnn_model.predict(X)
y_pred = (pred_pure_cnn.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by Pure CNN model: {correct}")
print(f"Accuracy of Pure CNN model: {acc:.2f}%")

2/2 [==============================] - 0s 16ms/step
Total no. of original gravitational data samples: 48
No. of correct predictions by Pure CNN model: 48
Accuracy of Pure CNN model: 100.00%


In [51]:
pred_cnn_transormer = cnn_transormer_model.predict(X)
y_pred = (pred_cnn_transormer.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by CNN + Transormfer model: {correct}")
print(f"Accuracy of CNN + Transormfer model: {acc:.2f}%")

2/2 [==============================] - 0s 6ms/step
Total no. of original gravitational data samples: 48
No. of correct predictions by CNN + Transormfer model: 37
Accuracy of CNN + Transormfer model: 77.08%


In [52]:
#
#
#
#Ensemble technique implementation
#
#
#

ensemble_pred = (pred_residual_cnn.ravel() + pred_cnn_gru.ravel() + pred_pure_cnn.ravel() + pred_cnn_transormer.ravel()) / 4
final_pred = (ensemble_pred.ravel() > 0.5).astype(int)
correct_final_pred = np.sum(final_pred == y)
acc_final = (correct_final_pred / len(y)) * 100

print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions: {correct_final_pred}")
print(f"Accuracy: {acc_final:.2f}%")

Total no. of original gravitational data samples: 48
No. of correct predictions: 48
Accuracy: 100.00%


In [32]:
import pandas as pd

data_BBH_df = pd.read_csv("./BBH/original-noise-unseen.csv", header=None)
X = data_BBH_df.values.astype(float)

print(len(data_BBH_df.index))

y = [0 for i in range(len(data_BBH_df.index))]

X = scaler.transform(X)

X = np.expand_dims(X, axis=-1)
print(X.shape)

48
(48, 16384, 1)


In [39]:
pred_residual_cnn = residual_cnn_model.predict(X)
y_pred = (pred_residual_cnn.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by Residual CNN model: {correct}")
print(f"Accuracy of Residual CNN model: {acc:.2f}%")

2/2 [==============================] - 0s 253ms/step
Total no. of original noise data samples: 48
No. of correct predictions by Residual CNN model: 48
Accuracy of Residual CNN model: 100.00%


In [40]:
pred_cnn_gru = cnn_gru_model.predict(X)
y_pred = (pred_cnn_gru.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by CNN + GRU model: {correct}")
print(f"Accuracy of CNN + GRU model: {acc:.2f}%")

2/2 [==============================] - 0s 53ms/step
Total no. of original noise data samples: 48
No. of correct predictions by CNN + GRU model: 48
Accuracy of CNN + GRU model: 100.00%


In [41]:
pred_pure_cnn = pure_cnn_model.predict(X)
y_pred = (pred_pure_cnn.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by Pure CNN model: {correct}")
print(f"Accuracy of Pure CNN model: {acc:.2f}%")

2/2 [==============================] - 0s 94ms/step
Total no. of original noise data samples: 48
No. of correct predictions by Pure CNN model: 48
Accuracy of Pure CNN model: 100.00%


In [42]:
pred_cnn_transormer = cnn_transormer_model.predict(X)
y_pred = (pred_cnn_transormer.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by CNN + Transormfer model: {correct}")
print(f"Accuracy of CNN + Transormfer model: {acc:.2f}%")

2/2 [==============================] - 0s 8ms/step
Total no. of original noise data samples: 48
No. of correct predictions by CNN + Transormfer model: 48
Accuracy of CNN + Transormfer model: 100.00%


In [43]:
#
#
#
#Ensemble technique implementation
#
#
#

ensemble_pred = (pred_residual_cnn.ravel() + pred_cnn_gru.ravel() + pred_pure_cnn.ravel() + pred_cnn_transormer.ravel()) / 4
final_pred = (ensemble_pred.ravel() > 0.5).astype(int)
correct_final_pred = np.sum(final_pred == y)
acc_final = (correct_final_pred / len(y)) * 100

print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions: {correct_final_pred}")
print(f"Accuracy: {acc_final:.2f}%")

Total no. of original noise data samples: 48
No. of correct predictions: 48
Accuracy: 100.00%
